In [27]:
import pandas as pd
import itertools
from scipy import stats

In [3]:
def get_gt(filename):
    with open(filename, "r") as fileHandler:
        lines = fileHandler.readlines()
    return lines

def get_pop(filename):
    phenotypes = []
    ids = []
    with open(filename, "r") as fileHandler:
        lines = fileHandler.readlines()
    for i in range(len(lines)):
        ids.append(lines[i].split()[0])
        phenotypes.append(lines[i].split()[2])
    return ids, phenotypes

def get_snp_ids(filename):
    snp_ids = ['IND', 'X', 'X', 'X', 'X', 'Pheno']
    with open(filename, "r") as fileHandler:
        lines = fileHandler.readlines()
    for i in range(len(lines)):
        snp_ids.append(lines[i].split()[1])
        snp_ids.append(lines[i].split()[1])
    return snp_ids

def get_ids(genotypes):
    ids = []
    for i in range(0, len(genotypes)):
        ids.append(genotypes[i].split()[0])
    return ids

def get_data_frame(genotypes, snp_ids):
    inds = []
    for i in range(0, len(genotypes)):
        inds.append(genotypes[i].split())
    df = pd.DataFrame(inds)
    df.columns = snp_ids
    return df

def which_minor_al(frq1, frq2, a1, a2):
    if frq1 < frq2:
        return frq1, a1
    else:
        return frq2, a2

def get_al_freq(snp_id, pop):
    aa = 0
    bb = 0
    ab = 0
    total = 2*len(pop[[snp_id]])
    al_1 = "1"
    al_2 = "2"

    for i in range(len(pop[[snp_id]])):
        
        if pop[[snp_id]].iloc[i].iloc[0] == '2' and pop[[snp_id]].iloc[i].iloc[1] == '2':
            aa += 1
        elif pop[[snp_id]].iloc[i].iloc[0] == '1' and pop[[snp_id]].iloc[i].iloc[1] == '1':
            bb += 1
        else:
            ab += 1

    obs_a = 2*aa + ab
    obs_b = 2*bb + ab

    f1 = (2*aa + ab) / total
    f2 = (2*bb + ab) / total

    return which_minor_al(f1, f2, al_1, al_2)

def get_missing(snp_id, df):
    missing_nu = 0
    for i in range(len(df[[snp_id]])):
        #if df[[snp_id]].iloc[i].all() == '0':
         #   missing_nu += 2
        if df[[snp_id]].iloc[i].any() == '0':
            missing_nu += 1
    return missing_nu, missing_nu / len(df[[snp_id]])

def missing_snps(person, nu_snps):
    missing = 0
    for i in range(nu_snps-6):
        print(i+6)
        if person[i+6] == '0' or person[i+7] == '0':
            missing += 1
        i += 2
    return missing

In [7]:
def chi_square_test(pop):
    aa_aff = 0
    bb_aff = 0
    ab_aff = 0

    aa_unaf = 0
    bb_unaf = 0
    ab_unaf = 0

    AFFECTED = '2'
    CONTROL = '1'

    for i in range(len(pop)):
        if pop.iloc[i].iloc[1] == '2' and pop.iloc[i].iloc[2] == '2' and pop.iloc[i].iloc[0] == AFFECTED:
            aa_aff += 1
        elif pop.iloc[i].iloc[1] == '1' and pop.iloc[i].iloc[2] == '1' and pop.iloc[i].iloc[0] == AFFECTED:
            bb_aff += 1
        elif pop.iloc[i].iloc[0] == AFFECTED:
            ab_aff += 1
        
        if pop.iloc[i].iloc[1] == '2' and pop.iloc[i].iloc[2] == '2' and pop.iloc[i].iloc[0] == CONTROL:
            aa_unaf += 1
        elif pop.iloc[i].iloc[1] == '1' and pop.iloc[i].iloc[2] == '1' and pop.iloc[i].iloc[0] == CONTROL:
            bb_unaf += 1
        elif pop.iloc[i].iloc[0] == CONTROL:
            ab_unaf += 1

    a = 2*aa_aff + ab_aff
    b = 2*bb_aff + ab_aff
    c = 2*aa_unaf + ab_unaf
    d = 2*bb_unaf + ab_unaf

    total = a + b + c + d

    #print (a, b, c, d)
    a_aff = a / total
    b_aff = b / total
    a_unaf = c / total
    b_unaf = d / total
    #print(a_aff, b_aff, a_unaf, b_unaf)

    p_aff = a_aff + b_aff
    p_unaf = a_unaf + b_unaf
    p_a = a_aff + a_unaf
    p_b = b_aff + b_unaf
    #print(p_aff, p_unaf, p_a, p_b)

    p_a_aff = p_aff * p_a
    p_b_aff = p_aff * p_b
    p_a_unaf = p_unaf * p_a
    p_b_unaf = p_unaf * p_b
    #print(p_a_aff, p_b_aff, p_a_unaf, p_b_unaf)

    exp_a_aff = total * p_a_aff
    exp_b_aff = total * p_b_aff
    exp_a_unaf = total * p_a_unaf
    exp_b_unaf = total * p_b_unaf
    #print(exp_a_aff, exp_b_aff, exp_a_unaf, exp_b_unaf)

    chi_a_aff = ((a - exp_a_aff) ** 2) / exp_a_aff
    chi_b_aff = ((b - exp_b_aff) ** 2) / exp_b_aff
    chi_a_unaf = ((c - exp_a_unaf) ** 2) / exp_a_unaf
    chi_b_unaf = ((d - exp_b_unaf) ** 2) / exp_b_unaf

    x_2 = chi_a_aff + chi_b_aff + chi_a_unaf + chi_b_unaf

    p_val = 1- stats.chi2.cdf(x_2, 1)
    print('Chi-square test (1 df): ', x_2)  
    print('P-value: ', p_val)

In [12]:

pop_file = pd.read_csv("pop.phe", sep='\s+', header=None)
pop_file.head()


,0,1,2
0,HCB181,1,1
1,HCB182,1,1
2,HCB183,1,1
3,HCB184,1,1
4,HCB185,1,1


In [ ]:
# getting 
ids = pop_file[0]
pop =  pop_file[2]


In [28]:
snp_ids = pd.read_csv("hapmap1.map", sep='\s+', header=None)[1]
snp_ids_diploid = [(x+"_1", x+"_2") for x in list(snp_ids)]
snp_ids_diploid = list(itertools.chain(*snp_ids_diploid))
snp_ids_diploid[0:10]

['rs6681049_1',
 'rs6681049_2',
 'rs4074137_1',
 'rs4074137_2',
 'rs7540009_1',
 'rs7540009_2',
 'rs1891905_1',
 'rs1891905_2',
 'rs9729550_1',
 'rs9729550_2']

In [17]:
genotype = pd.read_csv("hapmap1.ped", sep="\s+", header=None)
genotype['populations'] = pop
genotype.iloc[:10, :10]


In [32]:
col_names = ["individual", "X", "X", "X", "X", "phenotype"] + snp_ids_diploid

In [36]:
genotype.columns = col_names
genotype.iloc[:10, :10]

,individual,X,X,X,X,phenotype,rs6681049_1,rs6681049_2,rs4074137_1,rs4074137_2
0,HCB181,1,0,0,1,1,2,2,2,2
1,HCB182,1,0,0,1,1,2,2,1,2
2,HCB183,1,0,0,1,2,2,2,1,2
3,HCB184,1,0,0,1,1,2,2,1,2
4,HCB185,1,0,0,1,1,2,2,1,2
5,HCB186,1,0,0,1,1,2,2,2,2
6,HCB187,1,0,0,1,1,2,2,2,2
7,HCB188,1,0,0,1,1,2,2,1,2
8,HCB189,1,0,0,1,1,2,2,2,2
9,HCB190,1,0,0,1,1,2,2,2,2


In [ ]:
#### Heer resort the column headers so that:
# 1- Individual is your index
# 2- pop and phenotype are first columns, followed by snp_ids
# 3- remove the x columns if there are not needed or rename them to their proper names if they are needed.

In [43]:

df_pheno1 = genotype.loc[genotype['phenotype'] == 1]
df_pheno2 = genotype.loc[genotype['phenotype'] == 2]
df_pheno1.head()


,individual,X,X,X,X,phenotype,rs6681049_1,rs6681049_2,rs4074137_1,rs4074137_2,...,rs2269380_2,rs6151412_1,rs6151412_2,rs11912064_1,rs11912064_2,rs1001469_1,rs1001469_2,rs756638_1,rs756638_2,populations
0,HCB181,1,0,0,1,1,2,2,2,2,...,2,2,2,2,2,1,2,2,2,1
1,HCB182,1,0,0,1,1,2,2,1,2,...,2,2,2,2,2,1,2,2,2,1
3,HCB184,1,0,0,1,1,2,2,1,2,...,2,2,2,2,2,1,1,1,2,1
4,HCB185,1,0,0,1,1,2,2,1,2,...,2,2,2,2,2,1,2,2,2,1
5,HCB186,1,0,0,1,1,2,2,2,2,...,2,2,2,2,2,1,2,2,2,1


In [48]:
df_pop1 = genotype.loc[genotype['populations'] == 1]
df_pop2 = genotype.loc[genotype['populations'] == 2]
df_pop1.head()

,individual,X,X,X,X,phenotype,rs6681049_1,rs6681049_2,rs4074137_1,rs4074137_2,...,rs2269380_2,rs6151412_1,rs6151412_2,rs11912064_1,rs11912064_2,rs1001469_1,rs1001469_2,rs756638_1,rs756638_2,populations
0,HCB181,1,0,0,1,1,2,2,2,2,...,2,2,2,2,2,1,2,2,2,1
1,HCB182,1,0,0,1,1,2,2,1,2,...,2,2,2,2,2,1,2,2,2,1
2,HCB183,1,0,0,1,2,2,2,1,2,...,1,2,2,2,2,2,2,1,2,1
3,HCB184,1,0,0,1,1,2,2,1,2,...,2,2,2,2,2,1,1,1,2,1
4,HCB185,1,0,0,1,1,2,2,1,2,...,2,2,2,2,2,1,2,2,2,1


In [56]:
snp_id = 'rs6681049'
# maf_pop1, allele_pop1= get_al_freq('rs6681049', df_pop1)

snp_genotype = genotype[snp_id+"_1"].astype('str') +  + genotype[snp_id+"_2"].astype('str')

# usee pands counts number of occurrences
Counter(list(snp_genotype)) / sum(Counter (list(snp_genotype)).vales())

Counter({'11': 8, '12': 22, '22': 59})

In [ ]:
def get_al_freq(snp_id, pop):
    aa = 0
    bb = 0
    ab = 0
    //total = 2*len(pop[[snp_id]])
    
    
    
    al_1 = 1
    al_2 = 2

    for i in range(len(pop[[snp_id]])):
        
        if pop[[snp_id]].iloc[i].iloc[0] == '2' and pop[[snp_id]].iloc[i].iloc[1] == '2':
            aa += 1
        elif pop[[snp_id]].iloc[i].iloc[0] == '1' and pop[[snp_id]].iloc[i].iloc[1] == '1':
            bb += 1
        else:
            ab += 1

    obs_a = 2*aa + ab
    obs_b = 2*bb + ab

    f1 = (2*aa + ab) / total
    f2 = (2*bb + ab) / total

    return which_minor_al(f1, f2, al_1, al_2)


In [8]:







maf_pop2, allele_pop2 = get_al_freq('rs6681049', df_pop2)
phe_unaf_frq, allele_phe1 = get_al_freq('rs6681049', df_pheno1)
phe_af_frq, allele_phe2 = get_al_freq('rs6681049', df_pheno2)

# Missing statistics
missing_nu, freq = get_missing('rs6704013', df)
print('rs6704013: # of missing: ', missing_nu, ' frequency: ', freq)

missing_nu, freq = get_missing('rs307347', df)
print('rs307347: # of missing: ', missing_nu, ' frequency: ', freq)

missing_nu, freq = get_missing('rs9439440', df)
print('rs9439440: # of missing: ', missing_nu, ' frequency: ', freq)

# Minor allele statistics
print()
print('SNP ID: rs6681049')
print('MAF Population 1: ', maf_pop1, "Minor allele: ", allele_pop1)
print('MAF Population 2: ', maf_pop2, "Minor allele: ", allele_pop2)
print('Frequency in unaffected: ', phe_unaf_frq, "Allele: ", allele_phe1)
print('Frequency in affected: ', phe_af_frq, "Allele: ", allele_phe2)

# Basic Association Test
print()
df_snp1 = df[['Pheno', 'rs6681049']]
df_snp2 = df[['Pheno', 'rs9585021']]
df_snp3 = df[['Pheno', 'rs2222162']]

print('Basic Association Test rs6681049: ')
chi_square_test(df_snp1)

print()
print('Basic Association Test rs9585021: ' )
chi_square_test(df_snp2)

print()
print('Basic Association Test rs2222162: ')
chi_square_test(df_snp3)


rs6704013: # of missing:  2  frequency:  0.02247191011235955
rs307347: # of missing:  12  frequency:  0.1348314606741573
rs9439440: # of missing:  2  frequency:  0.02247191011235955

SNP ID: rs6681049
MAF Population 1:  0.23333333333333334 Minor allele:  2
MAF Population 2:  0.19318181818181818 Minor allele:  2
Frequency in unaffected:  0.26666666666666666 Allele:  2
Frequency in affected:  0.1590909090909091 Allele:  2

Basic Association Test rs6681049: 
Chi-square test (1 df):  3.066637047163362
P-value:  0.07991463136564891

Basic Association Test rs9585021: 
Chi-square test (1 df):  20.27001965692306
P-value:  6.7246097376072456e-06

Basic Association Test rs2222162: 
Chi-square test (1 df):  20.514418590898156
P-value:  5.918374757829525e-06


Note: The test for rs6958021 does not match the result in PLINK but the other SNPs do.